# Negative Weighted Events

In this notebook, we adapt the negative events-based measure from van den Broucke et al. from 2014 in the paper: "Determining Process Model Precision and Generalization with Weighted Artificial Negative Events" (doi: 10.1109/TKDE.2013.130). <br>
With respect to the actual executions of a process, the measure focuses on negative events, where a negative event represents information about activities that were prevented from taking place in the first place. Since they are rarely recorded in reality, we induce them into the log artificially. <br>
So, we basically induce all elements that weren't fired at this specific position in the event log. For simplicity, the authors assume that all other events, outside the current event itself, are inserted at each position in the trace. Afterwards, we check which of these events could be fired at the current position in the corresponding position or not. If an event can be fired, we increase a counter for allowed generalizations $AG$ by one, and if not, we increase a counter for allowed generalizations $DG$ by one. <br>
The final measure is calculated as follows: <br>
For event log $E$ and process model $M$,
$$ Generalization (L,M) = AG\,/\, (AG+DG).$$
One has to be aware, that this is not the complete logic of the paper, which is implemented in a further step by introducing a scoring mechanism. This is done due to the fact that we have to assume the completeness of an event log to induce negative events and this is normally not true in reality, as an event log only represents a subset. The scoring mechanism will allow us to loosen this assumption by only assuming the completeness property on a small window before the execution of the current activity.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.utils import get_happy_path_log, create_flower_model, generate_variant_model
from ocpa.objects.log.importer.csv import factory as ocel_import_factory_csv
from models.negative_events_measure import negative_events_without_weighting
import pickle
import multiprocessing
import numpy as np
from tqdm import tqdm
from models.negative_events_measure_parallel import process_group_without, negative_events_without_weighting_parallel

# O2C Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [3]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [4]:
value = negative_events_without_weighting (ocel, ocpn)
value

Check the arcs: 100%|██████████| 46/46 [00:00<00:00, 46312.53it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 170.00it/s]


0.5063

### Happy Path Petri Net

In [5]:
happy_path__ocel = get_happy_path_log(filename)

In [6]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [7]:
value = negative_events_without_weighting (ocel, happy_path_ocpn)
value

Check the arcs: 100%|██████████| 26/26 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 241.53it/s]


0.3073

### Flower Model Petri Net

In [8]:
ots = ["order","item","delivery"]

In [9]:
flower_ocpn = create_flower_model(filename,ots)

In [10]:
value = negative_events_without_weighting (ocel, flower_ocpn)
value

Check the arcs: 100%|██████████| 32/32 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 223.43it/s]


0.965

### Variant Model Petri Net

Import the primarily generated variant log for our measure computation, while we generate the variant model with the original log.

In [11]:
filename_variant = "../src/data/csv/order_process_variant_log.csv" 
object_types = ["order","item","delivery"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [12]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ots = ["order","item","delivery"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/order_variants/order_variant',object_types = ots,save_path_visuals=f"../reports/figures/order_variant_total.svg" )

Processing Variant Nets: 100%|██████████| 12/12 [00:00<00:00, 9409.54it/s]

#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


In [13]:
value = negative_events_without_weighting (ocel_variant, variant_ocpn)
value

Check the arcs: 100%|██████████| 378/378 [00:00<00:00, 17582.47it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:02<00:00, 19.65it/s]


0.1375

# P2P Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [14]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [15]:
value = negative_events_without_weighting (ocel, ocpn)
value

Check the arcs: 100%|██████████| 40/40 [00:00<00:00, 77065.76it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 838.91it/s]


0.1845

### Happy Path Petri Net

In [16]:
happy_path__ocel = get_happy_path_log(filename)

In [17]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [18]:
value = negative_events_without_weighting (ocel, happy_path_ocpn)
value

Check the arcs: 100%|██████████| 38/38 [00:00<00:00, 266974.12it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 781.10it/s]


0.1958

### Flower Model Petri Net

In [19]:
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]

In [20]:
flower_ocpn = create_flower_model(filename,ots)

In [21]:
value = negative_events_without_weighting (ocel, flower_ocpn)
value

Check the arcs: 100%|██████████| 38/38 [00:00<00:00, 38212.31it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 732.80it/s]


0.8611

### Variant Model Petri Net

Import the primarily generated variant log for our measure computation, while we generate the variant model with the original log.

In [22]:
filename_variant = "../src/data/csv/p2p_variant_log.csv" 
object_types = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [23]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/p2p_variants/p2p_variant',object_types = ots ,save_path_visuals=f"../reports/figures/p2p_variant_total.svg" )

Processing Variant Nets: 100%|██████████| 20/20 [00:00<00:00, 9618.86it/s]

#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


In [24]:
value = negative_events_without_weighting (ocel_variant, variant_ocpn)
value

Check the arcs: 100%|██████████| 760/760 [00:00<00:00, 21810.51it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 80.15it/s]


0.115

# BPI-Challenge 2017 Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [25]:
filename = "../src/data/jsonocel/BPI2017-Final.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [26]:
value = negative_events_without_weighting (ocel, ocpn)
value

Check the arcs: 100%|██████████| 120/120 [00:00<00:00, 118650.75it/s]
Calculate Generalization for all process executions: 100%|██████████| 31509/31509 [01:00<00:00, 524.52it/s]


0.3569

### Happy Path Petri Net

In [27]:
happy_path__ocel = get_happy_path_log(filename)

In [28]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [29]:
value = negative_events_without_weighting (ocel, happy_path_ocpn)
value

Check the arcs: 100%|██████████| 26/26 [00:00<00:00, 24363.70it/s]
Calculate Generalization for all process executions: 100%|██████████| 31509/31509 [01:00<00:00, 522.93it/s]


0.1077

### Flower Model Petri Net

In [30]:
ots = ["application","offer"]

In [31]:
flower_ocpn = create_flower_model(filename,ots)

In [32]:
value = negative_events_without_weighting (ocel, flower_ocpn)
value

Check the arcs: 100%|██████████| 56/56 [00:00<00:00, 19040.29it/s]
Calculate Generalization for all process executions: 100%|██████████| 31509/31509 [01:04<00:00, 487.77it/s]


0.8827

### Variant Model Petri Net

We import the pickle file for the variant model of the bpi challenge that we generated in the process models notebook.

In [5]:
filename_variant = "../src/data/csv/bpi2017_variant_log.csv" 
object_types = ["application","offer"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [6]:
with open("../src/data/csv/bpi_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

In [ ]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(ocel_variant, variant_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(6)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))

Check the arcs:  72%|███████▏  | 154697/214724 [2:22:02<46:37, 21.45it/s]  

# DS3 Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [3]:
filename = "../src/data/jsonocel/DS3.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [4]:
value = negative_events_without_weighting(ocel,ocpn)
value

Check the arcs: 100%|██████████| 130/130 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 4825/4825 [00:11<00:00, 405.91it/s] 


0.3701

### Happy Path Petri Net

In [5]:
happy_path__ocel = get_happy_path_log(filename)

In [6]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [7]:
value = negative_events_without_weighting(ocel,happy_path_ocpn)
value

Check the arcs: 100%|██████████| 8/8 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 4825/4825 [00:12<00:00, 395.43it/s] 


0.2073

### Flower Model Petri Net

In [8]:
ots = ["incident","customer"]

In [9]:
flower_ocpn = create_flower_model(filename,ots)

In [10]:
value = negative_events_without_weighting(ocel,flower_ocpn)
value

Check the arcs: 100%|██████████| 20/20 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 4825/4825 [00:12<00:00, 373.28it/s] 


0.9588

### Variant Model Petri Net

Import the primarily generated variant log for our measure computation, while we generate the variant model with the original log.

In [11]:
filename_variant = "../src/data/csv/DS3_variant_log.csv" 
object_types = ["incident","customer"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [12]:
with open("../src/data/csv/ds3_variant.pickle", 'wb') as fp:
    pickle.dump(ocel_variant, fp)

In [4]:
with open("../src/data/csv/DS3_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

In [5]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(ocel_variant, variant_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(7)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))

Check the arcs:  67%|██████▋   | 98030/146804 [47:10<29:01, 28.00it/s]  

# DS4 Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [3]:
filename = "../src/data/jsonocel/DS4.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [4]:
value = negative_events_without_weighting(ocel,ocpn)
value

Check the arcs: 100%|██████████| 364/364 [00:00<00:00, 168271.43it/s]
Calculate Generalization for all process executions: 100%|██████████| 14507/14507 [05:39<00:00, 42.69it/s]


0.3604

### Happy Path Petri Net

In [5]:
happy_path__ocel = get_happy_path_log(filename)

In [6]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [7]:
value = negative_events_without_weighting(ocel,happy_path_ocpn)
value

Check the arcs: 100%|██████████| 70/70 [00:00<00:00, 72512.05it/s]
Calculate Generalization for all process executions: 100%|██████████| 14507/14507 [05:07<00:00, 47.22it/s]


0.0842

### Flower Model Petri Net

In [8]:
ots = ["Payment application","Control summary","Entitlement application","Geo parcel document","Inspection","Reference alignment"]

In [9]:
flower_ocpn = create_flower_model(filename,ots)

In [10]:
value = negative_events_without_weighting(ocel,flower_ocpn)
value

Check the arcs: 100%|██████████| 162/162 [00:00<00:00, 23182.44it/s]
Calculate Generalization for all process executions: 100%|██████████| 14507/14507 [18:09<00:00, 13.31it/s]


0.6885

### Variant Model Petri Net

Import the primarily generated variant log for our measure computation, while we generate the variant model with the original log.

In [ ]:
filename_variant = "../src/data/csv/DS4_variant_log.csv" 
object_types = ["Payment application","Control summary","Entitlement application","Geo parcel document","Inspection","Reference alignment"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [ ]:
with open("../src/data/csv/DS4_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

In [ ]:
if __name__ == '__main__':
    
    #generate the variables needed for the parallel processing
    grouped_df, filtered_preceding_events_full, filtered_preceding_events, filtered_succeeding_activities_updated, events, silent_transitions = negative_events_without_weighting_parallel(ocel_variant, variant_ocpn)

    DG = 0  # Disallowed Generalization initialisation
    AG = 0  # Allowed Generalization initialisation

    # Create a multiprocessing Pool
    pool = multiprocessing.Pool(7)

    # Prepare the arguments for parallel processing
    args = [(group_key, df_group, filtered_preceding_events_full, filtered_preceding_events,
             filtered_succeeding_activities_updated, events, silent_transitions, AG, DG)
            for group_key, df_group in grouped_df]

    
    # Apply the parallel processing to each group with additional variables
    results = []
    with tqdm(total=len(grouped_df)) as pbar:
        for result in pool.imap_unordered(process_group_without, args):
            results.append(result)
            pbar.update(1)

    # Calculate the final sums of AG and DG
    final_AG = sum([result[0] for result in results])
    final_DG = sum([result[1] for result in results])

    # Close the multiprocessing Pool and join the processes
    pool.close()
    pool.join()

    # calculate the generalization based on the paper
    generalization = final_AG / (final_AG + final_DG)
    print(np.round(generalization,4))